# 1. precocessiong of mutation data

In [ ]:
#set path
setwd('/data/shanlan/crc_trio/1.sub_analysis')
out_dir='/data/shanlan/crc_trio/1.sub_analysis/plot/'
#create dir
dir.create("./runtime")
dir.create("./runtime/2.sgWGS")
dir.create("./runtime/2.sgWGS/tomaf")
dir.create("./runtime/2.sgWGS/maf")
dir.create("./runtime/2.sgWGS/ccf")
dir.create("./runtime/2.sgWGS/output")

In [ ]:
#set path
setwd('/data/shanlan/crc_trio/1.sub_analysis')
out_dir='/data/shanlan/crc_trio/1.sub_analysis/plot/'

#load library
#basic
library(openxlsx)
library(reshape2)
library(dplyr)
library(stringr)
library(car)

#visualization
library(ggpubr)
library(ggrepel)
library(ggpmisc)
library(ggsci)
library(cowplot)

#speciafic
library(ComplexHeatmap)
library(clusterProfiler)
library(maftools)
library(MesKit)
library(org.Hs.eg.db)
library(BSgenome.Hsapiens.UCSC.hg38)

In [ ]:
vcf_dir="/data/shanlan/crc_trio/wgsCrypt/results/2.VariantFilter/output"
#1.0 data load
filelist<-list.files(path=vcf_dir, pattern = "somatic.snv.filter.PASS.annovar.all.txt", full.names = TRUE)
#filelist=filelist[grepl('B139',filelist)]
length(filelist)
datalist <- lapply(filelist, function(x)read.csv(x, sep="\t",header=T))
#samples = read.table("./wgsCrypt_sample_pairs.txt", sep="\t") #该文件与sample_pairs.txt内容一样

name=gsub(paste0(vcf_dir,"/|_somatic.snv.filter.PASS.annovar.all.txt"),"",filelist)
names(datalist)=name

In [ ]:
#Colon_driver_genes.txt: drivergene_list(n=346) from Prof.Hu. 
#"where the putative driver genes were curated by merging all TCGA pan-cancer drivers [refs] (n = 299) and additional CRC-specific drivers (n = 47) annotated by the COSMIC Cancer Gene Census (https://cancer.sanger.ac.uk/census)."
#ref: Bailey, et al. Comprehensive Characterization of Cancer Driver Genes and Mutations. 和COSMIC
dr1 <- read.table("/data/shanlan/crc_trio/1.sub_analysis/refdb/genelist/Colon_driver_genes.txt",sep = "\t",header = T)$Gene #n=346, 
dr2 <- read.table("/data/shanlan/crc_trio/1.sub_analysis/refdb/genelist/zq_nc_crcdriver.txt",sep = "\t",header = F)$V1 #n=46, NC2021, Zhao qi
dr3 <- read.table("/data/shanlan/crc_trio/1.sub_analysis/refdb/Colon_precancer_genelist_STTT.txt",sep = '\t',header = T)$Gene ## n=21 , STTT2024, Shaosen Zhang, et al.
top20_crc_wes <- c('APC','TP53','KRAS','FBXW7','LRP1B','PIK3CA','FAT3','FAT4','PCDH17','SMAD4','SOX9','ZFP36L2','ARID1A','TCF7L2','SPTA1','ATM','BRAF','FAT2','MACF1','NBEA','ACVR2A','BCL9L') # n=21, T_wes>5%
driverGene <- Reduce(union,list(dr1,dr2,dr3,top20_crc_wes)) 
driverGene %>% length() #367
write.table(data.frame(Gene=driverGene,Tumor='Colon'),file = "/data/shanlan/crc_trio/1.sub_analysis/refdb/Colon_driver_final.txt",sep = '\t',quote = F, col.names = F, row.names = F )


In [ ]:
##rescue APC mutations across all crypts !!!!! # 进一步改进，对所有driver都recall
#apc_mut<-read.table('/data/shanlan/crc_trio/wgsCrypt/results/2.Mutect/Solo/sgWGS.Mutect2.APC.txt',header=T,sep='\t',fill=T)
#colnames(apc_mut)=gsub('FAP03_N1_B1_|FAP03_P1_R3_G4_','',colnames(apc_mut))

dr1<-read.table('/data/shanlan/crc_trio/wgsCrypt/results/2.Mutect/Solo/sgWGS.Mutect2.Driver.txt',header=T,sep='\t',fill=T) %>% mutate(mut_id = paste0(Sample, ":", Chr, ":", Start, ":", End, ":", Ref, ":", Alt))
dr2<-read.table('/data/shanlan/crc_trio/wgsCrypt/results/2.Strelka/sgWGS.Strealk2.Driver.txt',header=T,sep='\t',fill=T) %>% mutate(mut_id = paste0(Sample, ":", Chr, ":", Start, ":", End, ":", Ref, ":", Alt))
apc_all <- rbind(dr1, dr2 %>% filter(!mut_id %in% dr1$mut_id) ) %>% dplyr::select(-mut_id) %>% filter(Sample %in% names(datalist) )
colnames(apc_all)[7:12]=c("N_DP","N_VAF","Total_allele_depth","Ref_allele_depth","Alt_allele_depth","VAF")

#----------------------------------------------------------------------------------------------------
# remove false postive mutations
# criteria: (1)AD>=3 & DP>=5; or AD>=2 & VAF>=0.1
# (2) The site distance to adjacent mutations is greater than 50 bp.  
# Specially, 如果两个突变的VAF差距大，保留；另外，对连续插入或缺失需要手动检查
#----------------------------------------------------------------------------------------------------
tmp1 = apc_all %>% mutate(mut_id = paste0(Chr, ":", Start, ":", End, ":", Ref, ":", Alt)) 
tmp1=tmp1 %>% filter((Alt_allele_depth>2 & Total_allele_depth>=5) | (Alt_allele_depth>=2 & VAF>=0.1)) 

threshold = 50 
tmp2= tmp1 %>%  
              arrange(Sample,Chr,Start) %>% group_by(Sample,Chr) %>%  
              mutate(  
                distance_to_previous = Start - lag(Start),  
                distance_to_next = lead(Start) - Start,
                diffVAF_to_previous = as.numeric(VAF) - lag(as.numeric(VAF)),  
                diffVAF_to_next = lead(as.numeric(VAF)) - as.numeric(VAF),
                group = cumsum(is.na(distance_to_previous) | distance_to_previous > threshold),
                id = paste0("id", group), 
                LenRef = nchar(Ref),
                LenAlt = nchar(Alt) 
              ) %>%  
              mutate(  
                to_delete1 = ifelse(  
                  (!is.na(distance_to_previous) & distance_to_previous <= threshold) |  
                  (!is.na(distance_to_next) & distance_to_next <= threshold), 1, 0 ), 
                 to_delete2 = ifelse(  
                  (!is.na(distance_to_previous) & distance_to_previous <= 10) |  
                  (!is.na(distance_to_next) & distance_to_next <= 10), 1, 0), 
                 to_delete3 = ifelse(  
                  (!is.na(diffVAF_to_previous) & abs(diffVAF_to_previous) <= 0.15) |  
                  (!is.na(diffVAF_to_next) & abs(diffVAF_to_next) <= 0.15), 1, 0) 
                 
              ) 
#输出待手动检查突变
write.table(tmp2 %>% filter(to_delete1==1), "./runtime/2.sgWGS/apc_mut_near.txt", sep = "\t", quote=FALSE, row.names = FALSE)

#选择高可信突变
apc_hcv<- tmp2 %>% filter(to_delete1!=1)%>% ungroup() %>% dplyr::select(c(1:29)) #dplyr::select(-distance_to_previous, -distance_to_next, -to_delete1)
write.table(apc_hcv, "./runtime/2.sgWGS/apc_mut_hcv.txt", sep = "\t", quote=FALSE, row.names = FALSE)

#重新挽救的邻近突变
apc_rec <- read.table('/data/shanlan/crc_trio/1.sub_analysis/runtime/2.sgWGS/apc_mut_near_rescue.txt',header=T,sep='\t',fill=T)

#数据合并
apc_mut = rbind(apc_hcv,apc_rec)

write.table(apc_mut, "./runtime/2.sgWGS/rescued_apc_mut_final.txt", sep = "\t", quote=FALSE, row.names = FALSE)


In [26]:
###driver突变负荷
sn_info <- read.table('/data/shanlan/crc_trio/1.sub_analysis/runtime/2.sgWGS/mt/ST1.mt_sn_info.txt', header = T, sep = '\t')
df = apc_mut %>% left_join(.,sn_info, by = c('Sample'='Tumor_Sample_Barcode'))
tt = df %>% dplyr::select(Sample,Derived,Gene.refGene) %>% unique() %>% group_by(Gene.refGene,Derived) %>% tally() %>% arrange(desc(n))


In [ ]:
##1.1 Filtering false postive mutations, please rescue oncogenic APC mutations 
#运行前把annodir下所有文件都删除了
mafdir = '/data/shanlan/crc_trio/1.sub_analysis/runtime/2.sgWGS/tomaf/'
unlink(file.path(mafdir, "*"), recursive = FALSE)

dat=list()
for(i in 1:length(datalist)){
    tmp1=datalist[[i]] %>% mutate(mut_id = paste0(Chr, ":", Start, ":", End, ":", Ref, ":", Alt))
    colnames(tmp1)[6:11]=c("N_DP","N_VAF","Total_allele_depth","Ref_allele_depth","Alt_allele_depth","VAF")
    print(paste0(names(datalist)[i],":na:",nrow(tmp1 %>% filter(is.na(N_DP) ))) )
    #tmp1=tmp1%>% filter(N_DP >= 10 & Total_allele_depth >= 7 & Alt_allele_depth >= 4 & VAF >= 0.15)
    tmp1=tmp1%>% filter(VAF >= 0.15) #because of refiltered-samples with low depth
    
    tmp2=apc_mut %>% filter(Sample %in% names(datalist)[i]) %>% dplyr::select(-Sample) %>% mutate(mut_id = paste0(Chr, ":", Start, ":", End, ":", Ref, ":", Alt)) %>% filter(!mut_id %in% tmp1$mut_id)
    colnames(tmp2)=colnames(tmp1)
    
    tmp=rbind(tmp1,tmp2) %>% distinct(.keep_all = TRUE) %>% mutate(Tumor_Sample_Barcode=names(datalist)[i]) %>% dplyr::select(Tumor_Sample_Barcode, mut_id, everything())  
 
    tmp=tmp %>% 
     filter(is.na(ExAC_ALL) | ExAC_ALL<0.05 ) %>%
     filter(is.na(ExAC_EAS) | ExAC_EAS<0.05 ) %>%
     filter(is.na(X1000g2015aug_all) | X1000g2015aug_all<0.05 ) %>% 
     filter(is.na(esp6500siv2_all) | esp6500siv2_all<0.05 ) 
    
    dat[[i]]=tmp
    ##duplicate mutation info for each sample
    dup_mut=tmp %>% group_by(mut_id) %>% tally() %>% filter(n>1)
    print(paste0(names(datalist)[i],":dup:",nrow(dup_mut)))
}
names(dat)=names(datalist)
lapply(dat, function(x)write.table(x,paste("./runtime/2.sgWGS/tomaf/", x$Tumor_Sample_Barcode[1], ".txt",sep=""),sep="\t",quote=FALSE, row.names = FALSE))

##重新读入处理好的突变文件， for maftools，找高频的突变
toMaflist<-list.files(path="./runtime/2.sgWGS/tomaf", pattern = ".txt", full.names = TRUE)
#toMaflist<-toMaflist[grepl('B139',toMaflist)]
Maflist <- lapply(toMaflist, function(x)annovarToMaf(x,Center = 'NA', refBuild = 'hg38', tsbCol = 'Tumor_Sample_Barcode',table = 'refGene',sep = "\t"))
names(Maflist)=gsub("./runtime/2.sgWGS/tomaf/|.txt","",toMaflist)

In [ ]:
##重新读入处理好的突变文件， for maftools，找高频的突变
toMaflist<-list.files(path="./runtime/2.sgWGS/tomaf", pattern = ".txt", full.names = TRUE)
#toMaflist<-toMaflist[grepl('B139',toMaflist)]
Maflist <- lapply(toMaflist, function(x)annovarToMaf(x,Center = 'NA', refBuild = 'hg38', tsbCol = 'Tumor_Sample_Barcode',table = 'refGene',sep = "\t"))
names(Maflist)=gsub("./runtime/2.sgWGS/tomaf/|.txt","",toMaflist)
names(Maflist)
names(Maflist) %>% length()

In [ ]:
clinical<-data.frame(Tumor_Sample_Barcode= names(Maflist) ) %>% mutate(
    Patient = ifelse(grepl("^B",Tumor_Sample_Barcode),stringr::str_sub(Tumor_Sample_Barcode,1,4), 
                     stringr::str_sub(Tumor_Sample_Barcode,1,5) ),
    Tissue = ifelse(grepl("N",Tumor_Sample_Barcode),"N",
                    ifelse(grepl("^B",Tumor_Sample_Barcode),
                           stringr::str_sub(Tumor_Sample_Barcode,5,6) %>% gsub("_","",.), 
                           stringr::str_sub(Tumor_Sample_Barcode,7,8) ) ), 
    Tumor=paste0(Patient,Tissue),
    Tumor_Sample_Label = ifelse(grepl("^B",Tumor_Sample_Barcode),stringr::str_sub(Tumor_Sample_Barcode,5,-1), stringr::str_sub(Tumor_Sample_Barcode,7,-1) ),
    Label=ifelse(grepl("^B",Tumor_Sample_Barcode),stringr::str_sub(Tumor_Sample_Barcode,6+nchar(Tissue),-1), stringr::str_sub(Tumor_Sample_Barcode,10,-1) ),
    Region=stringr::str_sub(Label,1,2),
    Derived=substr(Tissue,1,1)
) %>% mutate(tmp=ifelse(grepl("^B",Tumor), Tumor, paste0(substr(Tumor,1,5),"_",substr(Tumor,6,nchar(Tumor) ))), Tumor=tmp ) %>% dplyr::select(-tmp)
write.table(clinical, "./runtime/2.sgWGS/sgWGS_clinical_all.txt", sep = "\t", quote=FALSE, row.names = FALSE)


In [ ]:
maf=Maflist                 
maf_flatten<-do.call("rbind", lapply(maf, as.data.frame)) %>%
             mutate_at(.vars = vars(N_DP, N_VAF, Total_allele_depth, Ref_allele_depth, Alt_allele_depth, VAF), .fun = as.numeric)
#maf_flatten$Chromosome<-as.character(maf_flatten$Chromosome)
write.table(maf_flatten, "./runtime/2.sgWGS/sgWGS_maf_all", sep = "\t", quote=FALSE, row.names = FALSE) #如果meskit输入有ccf文件，则maf文件不能包含CCF列，不然报错，错误: Problem with `mutate()` input `CCF`.

write.table(maf_flatten[which(maf_flatten$Variant_Type=="SNP"),], "./runtime/2.sgWGS/sgWGS_maf_all_SNP", sep = "\t", quote=FALSE, row.names = FALSE) #如果meskit输入有ccf文件，则maf文件不能包含CCF列，不然报错，错误: Problem with `mutate()` input `CCF`.

maf_flatten_exonic = subset(maf_flatten[which(maf_flatten$Variant_Type=="SNP"),],Variant_Classification == c("Missense_Mutation","Silent","Nonsense_Mutation"))
write.table(maf_flatten_exonic, "./runtime/2.sgWGS/sgWGS_maf_exonic_SNP", sep = "\t", quote=FALSE, row.names = FALSE)

In [ ]:
#--------------------------------------------------------------------------------------
# ###oldrevision: re-analysis for FAP01/FAP03, excluded FAP03_N2_R4_G6, filter lesions with sn<4
#2024-12-13: maf share with Christina
#Save the file format as before
#----------------------------------------------------------------------------------------
library(dplyr)
#black list: 综合多个条件来确定
black_list<-read.table("/data/shanlan/crc_trio/1.sub_analysis/sgWGS_black_list.txt", sep = "\t", header = F)$V1
black_list %>% sort()
black_list %>% length()

##exclude blacklist
maf=Maflist[!names(Maflist) %in% black_list] 
#maf=Maflist
maf_flatten<-do.call("rbind", lapply(maf, as.data.frame)) %>%
             mutate_at(.vars = vars(N_DP, N_VAF, Total_allele_depth, Ref_allele_depth, Alt_allele_depth, VAF), .fun = as.numeric) %>%
             dplyr::select(Chromosome, Start_Position, End_Position, Reference_Allele, Tumor_Seq_Allele2, Tumor_Sample_Barcode, everything()) 

shared_lesions=c('FAP01_N','FAP01_P1','FAP01_P2','FAP01_P4','FAP01_P5','FAP01_P6','FAP01_T2','FAP01_T3','FAP03_N','FAP03_P1','FAP03_P2','FAP03_P4','FAP03_P5')
shared_pts=c('FAP01','FAP03')
#for Ryan
TREEDIR="/data/shanlan/crc_trio/1.sub_analysis/runtime/2.sgWGS/tree/"
combn_files <- list.files(path=TREEDIR, pattern = "_clean_meskit", full.names = TRUE)
combn_files <- combn_files[grep('FAP01_|FAP03_',combn_files)]
combn_files

f_min= list()

for(i in combn_files ) {
    #i=1
    sn<- read.table(i,head=T,sep='\t')
    sn <- sn %>% filter(Tumor_Sample_Barcode != "FAP03_N2_R4_G6") %>% filter(!grepl('FAP01_P3|FAP01_T1|FAP03_P3|FAP03_P6|FAP03_P7|FAP03_P8',Tumor_Sample_Barcode) ) #|_B1
    objects<-gsub(paste0(TREEDIR,"/|_clinical_clean_meskit"),"",i)
    mut <- maf_flatten %>% filter(Tumor_Sample_Barcode %in% sn$Tumor_Sample_Barcode)
    f_min[[objects]] = mut %>% filter(VAF<0.15)
    write.table(mut, paste0(TREEDIR,objects,".all_maf_v2.txt"), sep = "\t", quote=FALSE, row.names = FALSE,col.names = TRUE) 
}



In [ ]:
##rescued list to CC
library(dplyr)
#black list: 综合多个条件来确定
black_list<-read.table("/data/shanlan/crc_trio/1.sub_analysis/sgWGS_black_list.txt", sep = "\t", header = F)$V1
black_list %>% sort()
black_list %>% length()
apc_mut<-read.table( "/data/shanlan/crc_trio/1.sub_analysis/runtime/2.sgWGS/rescued_apc_mut_final.txt", sep = "\t", header = T)
apc_mut %>% dim()

shared_lesions=c('FAP01_N','FAP01_P1','FAP01_P2','FAP01_P4','FAP01_P5','FAP01_P6','FAP01_T2','FAP01_T3','FAP03_N','FAP03_P1','FAP03_P2','FAP03_P4','FAP03_P5')
shared_pts=c('FAP01','FAP03')

apc_mut_toCC <- apc_mut %>% filter(!Sample %in% black_list) %>% filter(grepl(paste0(shared_lesions,collapse = "|"),Sample))
apc_mut_toCC %>% dim()
write.table(apc_mut_toCC, "/data/shanlan/crc_trio/1.sub_analysis/runtime/2.sgWGS/rescued_apc_mut_FAP.txt", sep = "\t", quote=FALSE, row.names = FALSE)